<a href="https://colab.research.google.com/github/Ajayraju7/Fmml-module-labs-projects/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [12]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.16678700361010831


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [13]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.340858347804515
Validation accuracy using random classifier is  0.16149838749689904


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [14]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.3460979156568105


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [15]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [9]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [10]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.3428815287458775
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1)Ans:
Averaging validation accuracy across multiple splits, as done in cross-validation, provides a more accurate estimate of how well your model is likely to perform on unseen data compared to a single validation split. However, it's important to note that this estimate is still not a direct substitute for the actual test accuracy on completely unseen data.

The purpose of cross-validation is to provide a better estimate of a model's generalization performance than a single validation split, and it can help in selecting models and hyperparameters that are more likely to perform well on unseen data. Nevertheless, the true test accuracy can only be determined by evaluating your model on a separate, held-out test dataset that it has never seen during training or validation.

In summary, cross-validation gives a more accurate estimate of performance than a single split, but it's not a replacement for evaluating your model on an independent test set to get the actual test accuracy

2)Ans:
True Test Accuracy: The true test accuracy is the accuracy of a machine learning model on a completely independent and unseen test dataset that was not used during training or cross-validation. It represents the model's performance in a real-world scenario.

Cross-Validation as an Estimate: Cross-validation provides a more accurate estimate of how well your model is likely to perform on unseen data compared to a single train-test split. This is because cross-validation systematically evaluates the model's performance across multiple data splits, providing a more robust and reliable assessment of its generalization capabilities.

Reduced Variability: Cross-validation helps reduce the impact of randomness in data splitting, which can occur in a single train-test split. By averaging the results of multiple train-test splits (folds), cross-validation provides a more stable and less biased estimate of model performance.

Hyperparameter Tuning and Model Selection: Cross-validation is often used during hyperparameter tuning and model selection processes. It allows you to compare different models and hyperparameters more accurately, helping you choose the best-performing model configuration.

3)Ans:
The number of iterations in cross-validation, often referred to as the number of folds, can have an impact on the estimate of a model's performance:

1. **Fewer Folds (Lower Iterations)**:
    Pros: Faster to compute as you're running the model fewer times.
    Cons: The estimate may have higher variability and be less reliable because each fold represents a smaller portion of the data, potentially leading to less robust results. It may not capture the dataset's variability as effectively.
2. More Folds
  Pros: Typically provides a more stable and reliable estimate of a model's performance because it averages results from a larger number of test sets, reducing the impact of random data splits.
  Cons: Takes more time to compute since you're training and evaluating the model multiple times, which can be computationally expensive with very large datasets.
In general, using a higher number of iterations in cross-validation is preferred for obtaining a better estimate of a model's performance. Common choices include 5-fold and 10-fold cross-validation, as they strike a balance between computation time and reliability. However, the optimal number of folds can vary depending on the specific dataset and problem. If computational resources allow, using even higher numbers of folds   can further improve the reliability of the estimate.

4)Ans:
Increasing the number of iterations (folds) in cross-validation can help to some extent in dealing with very small training or validation datasets, but it's not a panacea. While more iterations can provide a more stable estimate of performance, there are limitations to how much it can compensate for small dataset sizes:

1.Benefits of More Iterations
  With more iterations, you get a better average estimate of model performance, which can help mitigate some of the randomness associated with small dataset splits.
   It can improve the robustness of hyperparameter tuning and model selection, as you have more opportunities to assess model performance.

2. Limitations with Small Datasets
   Small training datasets may not provide enough information for the model to learn meaningful patterns, even with increased cross-validation iterations. You may still face issues with overfitting or underfitting.
   Small validation datasets can lead to noisy estimates of performance, and increasing the number of iterations won't magically create more validation data.
   
3.Computational Overhead Using a large number of iterations can be computationally expensive, especially if you have limited computational resources or a complex model.
In cases where you have very small datasets, it's crucial to consider other strategies, such as:

Data Augmentation If possible, you can artificially increase the size of your training dataset by applying data augmentation techniques, which generate new training examples from the existing ones.

Regularization Use appropriate regularization techniques (e.g., L1, L2 regularization) to prevent overfitting when dealing with small datasets.

Transfer LearningIf applicable, leverage pre-trained models or features learned on larger datasets and fine-tune them on your small dataset.

Collect More Data If feasible, consider acquiring additional data to increase the size of your training dataset